<a href="https://colab.research.google.com/github/chris-bhaila/ANAIS-2025/blob/main/Day_4_Agents_lab_ANAIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to Agents
In this lab, we will explore use of LLMs for building agentic system. We will do so by implementing a flight booking agent.

### 1. Environment Setup
The first step ensures the environment has the necessary tools to communicate with Google's servers.

**!pip install -q -U google-genai:** This command installs (or updates) the google-genai library.

In [2]:
!pip install -q -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.1/719.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 17.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.


#### **Important**
Please create your Google AI studion API keys as below :
- Visit https://aistudio.google.com/api-keys
- Create Project under "Choose an imported project" dropdown
- Name your key anything you want(Only after creating project)
- Choose create key
- Now you can find key under Key column of the project - click on the key valye to reveal api key
- Copy the key and paste below under "insert_api_key"


In [3]:
from google import genai
from google.genai import types
import re #Regular Expressions

#Initialize the client
client = genai.Client(api_key="google_api_key")
model_name = "gemma-3-27b-it" #Gemma 3 is a high performance open weight model with 27 billion parameters (it-Instruction Tuned, optimized for chat and following directions)

safety_settings = [
    types.SafetySetting(
        category = "HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold = "BLOCK_ONLY_HIGH",
    ),
]


### **Data Classes**

#### **1. `Date`**
Represents a date and time with four components:
- `year`: Integer year
- `month`: Integer month (1-12)
- `day`: Day of the month
- `hour`: Hour of the day (0-23)

This class is useful for storing precise timestamps for events such as scheduled flights.

---

#### **2. `UserProfile`**
Represents basic user information:
- `user_id`: A unique identifier for the user
- `name`: User's name
- `email`: User's email address

This structure can be used to store or process user accounts, bookings, or preferences.

---

#### **3. `Flight`**
Represents the details of a flight:
- `flight_id`: Flight identifier or number
- `origin`: Departure location
- `destination`: Arrival location
- `date_time`: A `Date` object representing when the flight occurs
- `duration`: Flight duration in minutes
- `price`: Cost of the flight (likely in some currency unit)

This structure organizes all relevant flight information into one clean object.

---



In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from dataclasses import dataclass
import json

@dataclass
class Date:
  year: int; month: int; day: int; hour: int

@dataclass
class UserProfile:
  user_id: str; name: str; email: str

@dataclass
class Flight:
  flight_id: str; origin: str; destination: str; date_time: Date; duration: int; price: int

## Populating our database

In [5]:
#USER DATABASE
user_database = {
    "Adam": UserProfile(user_id="1", name="Adam", email="adam@gmail.com"),
    "Bob": UserProfile(user_id="2", name="Bob", email="bob@gmail.com"),
    "Chelsie": UserProfile(user_id="3", name="Chelsie", email="chelsie@gmail.com"),
    "David": UserProfile(user_id="4", name="David", email="david@gmail.com"),
    "Eve": UserProfile(user_id="5", name="Eve", email="eve@outlook.com"),
    "Frank": UserProfile(user_id="6", name="Frank", email="frank@tech.com"),
    "Grace": UserProfile(user_id="7", name="Grace", email="grace@edu.org"),
    "Heidi": UserProfile(user_id="8", name="Heidi", email="heidi@web.de"),
}

#FLIGHT DATABASE
# Format: Flight(id, origin, destination, Date(Y, M, D, H), duration, price)
flight_database = {
    # SFO to JFK (Existing + New)
    "DA123": Flight("DA123", "SFO", "JFK", Date(2025, 9, 1, 1), 3, 200),
    "DA125": Flight("DA125", "SFO", "JFK", Date(2025, 9, 1, 7), 9, 500),
    "DA126": Flight("DA126", "SFO", "JFK", Date(2025, 9, 1, 13), 5, 350),
    "DA127": Flight("DA127", "SFO", "JFK", Date(2025, 9, 2, 8), 5, 250),

    # SFO to SNA (Existing + New)
    "DA456": Flight("DA456", "SFO", "SNA", Date(2025, 10, 1, 1), 2, 100),
    "DA460": Flight("DA460", "SFO", "SNA", Date(2025, 10, 1, 9), 2, 120),
    "DA461": Flight("DA461", "SFO", "SNA", Date(2025, 10, 2, 10), 2, 90),

    # LAX to ORD
    "DA701": Flight("DA701", "LAX", "ORD", Date(2025, 11, 5, 6), 4, 300),
    "DA702": Flight("DA702", "LAX", "ORD", Date(2025, 11, 5, 14), 4, 280),
    "DA703": Flight("DA703", "LAX", "ORD", Date(2025, 11, 6, 9), 4, 450),

    # SEA to MIA
    "DA801": Flight("DA801", "SEA", "MIA", Date(2025, 12, 10, 23), 6, 600),
    "DA802": Flight("DA802", "SEA", "MIA", Date(2025, 12, 11, 5), 6, 550),

    # BOS to SFO
    "DA901": Flight("DA901", "BOS", "SFO", Date(2025, 8, 15, 10), 6, 400),
    "DA902": Flight("DA902", "BOS", "SFO", Date(2025, 8, 15, 16), 6, 380),
}

#Reset dynamic databases for the lab session
itinerary_database = {}
ticket_database = {}

### Brief Explanation of the Tools

#### **get_user_info(name)**
Looks up a user by name in the `user_database` and returns their stored profile. Returns an error message if the user doesn't exist.

#### **fetch_flight_info(origin, destination)**
Searches `flight_database` for flights matching the given origin and destination. Returns a list of matching flights or an error if none are found.

#### **book_flight(user_name, flight_id)**
Books a flight for a user by linking their name to a selected flight in `itinerary_database`. Returns a confirmation message or an error if the flight ID is invalid.

#### **file_ticket(user_name, issue)**
Creates a customer support ticket with a unique ID and stores the issue in `ticket_database`. Returns a confirmation with the ticket ID.

#### **tools (mapping)**
A dictionary that maps tool names (strings) to their corresponding functions for easy lookup during agent operations.


In [7]:
#TOOLS

def get_user_info(name):
  """
  Retrieve user information from the database.

  This function looks up a user by their name and returns their complete
  profile including email, name, and user_id. This information is essential
  for booking flights and other operations requiring user identification,

  Args:
    name (str): The name of the user to look up.
  """

  return str(user_database.get(name, "Error: User not found."))


def fetch_flight_info(origin, destination):
  """
  Fetch available flights between two airports.

  This function queries the flight database to find all available flights
  from the origin airport to the destination airport. It returns detailed
  information about each matching flight including flight ID, airline,
  departure/arrival times and price.

  Args:
    origin(str): The IATA airport code for the departure city
    destination(str): The IATA airport code for the arrival city
  """

  flights = [f for f in flight_database.values()
            if f.origin == origin and f.destination == destination]
  print(flights)
  return str(flights) if flights else "Error: No flights found."

def book_flight(user_name, flight_id):
  """
  Book a specific flight for a user.

  This function creates a flight booking by associating a user with a specific
  flight. The booking is stored in the itinerary database, which can be
  retrieved later using fetch_itinerary().

  Args:
      user_name (str): The name of the user making the booking.
      flight_id (str): The unique identifier of the flight to book (e.g., "UA123").

  Returns:
      str: A confirmation message if booking is successful.
            Returns "Error: Invalid Flight ID." if the flight doesn't exist.
  """
  if flight_id not in flight_database:
      return "Error: Invalid Flight ID."
  itinerary_database[user_name] = flight_database[flight_id]
  return f"Confirmed: {flight_id} booked for {user_name}."


def file_ticket(user_name, issue):
    """
    File a customer support ticket for a user.

    This function creates a support ticket when a user reports an issue with
    their booking, flight, or any other service-related problem. Each ticket
    is assigned a unique identifier and stored in the ticket database for
    tracking and resolution.

    Args:
        user_name (str): The name of the user filing the ticket.
        issue (str): A description of the problem or issue being reported.

    Returns:
        str: A confirmation message containing the unique ticket ID (format: "TKT-N").
    """
    tid = f"TKT-{len(ticket_database) + 1}"
    ticket_database[tid] = {"user": user_name, "issue": issue}
    return f"Confirmed: Support ticket {tid} filed."

#Tool mapping for the agent loop
tools = {
    "get_user_info": get_user_info,
    "fetch_flight_info": fetch_flight_info,
    "book_flight": book_flight,
    "file_ticket": file_ticket
}

### Brief Explanation of `clean_arg(arg)`

This function cleans and normalizes a single argument string:

- Trims leading and trailing whitespace.  
- If the argument contains a keyword form like `name=value`, it keeps only the part after `=`.  
- Removes surrounding single or double quotes from the remaining value.

The result is a clean, plain argument string ready for further processing.


In [8]:
def clean_arg(arg):
  """
  Cleans a single argument:
  - Removes keyword(name=)
  - Removes surrounding quotes
  """
  arg = arg.strip()

  #If keyword argument is present, split on first '=' otherwise, it will send 'args=argument'
  if "=" in arg:
    arg = arg.split("=",1)[1]

  #Strip quotes and whitespace
  return arg.strip().strip("'").strip('"')

### Brief Explanation of the Agent Loop Code

#### **SYSTEM_PROMPT**
Defines strict rules the agent must follow:
- It acts as a flight booking assistant.
- It can use four tools (`get_user_info`, `fetch_flight_info`, `book_flight`, `file_ticket`).
- It must output **one action at a time** in the format:

- After each action, the agent must wait for an Observation.

---

### **run_agent_lab(user_query)**

This function simulates a step-by-step agent loop:

1. **Initialize conversation history** with the system prompt + user query.
2. **Iterate up to 6 steps**, generating a new LLM response each time.
3. Print the model's output for debugging.
4. **Parse the action** using a regex to detect:
5. **Clean the arguments** using `clean_arg()`.
6. **Execute the corresponding tool function** from the `tools` dictionary.
7. Print and append the resulting **Observation** to the history.
8. **Stop early** if a confirmation or error occurs (indicating a final state).
9. Ends the loop if no action is detected.

---

### **Test Case**
The last line runs the agent with:
> “Hi, I'm Bob. Book me the cheapest flight from LAX to ORD
”

This triggers the whole agent workflow for demonstration.


In [9]:
SYSTEM_PROMPT = """You are a Flight Booking Agent. Use the following tools:
1. get_user_info(name) : This gives the email, name and user_id of a user. This is important for booking the flight
2. fetch_flight_info(origin, destination)
3. book_flight(user_name, flight_id)
4. file_ticket(user_name, issue)

CRITICAL PROTOCOL:
- You MUST output ONLY ONE action at a time
- After outputting an Action, you MUST STOP and wait for the Observation
- DO NOT write the Observation yourself - it will be provided to you
- DO NOT perform multiple actions in one response

Format:
Thought: [Your reasoning about what to do next]
Action: tool_name(args)

That's it. Stop there and wait for the Observation.
"""

def run_agent_lab(user_query):
    history = SYSTEM_PROMPT + f"\n\nUser: {user_query}"

    for i in range(6):  # Limit steps for safety
        response = client.models.generate_content(
            model=model_name,
            contents=history,
            config=types.GenerateContentConfig(
                temperature=0.1,
                safety_settings=safety_settings
            )
        )

        output = response.text
        print(f"\033[94m-- Step {i+1} --\033[0m\n{output}")


        # Parse Action: tool_name(arg1, arg2)
        match = re.search(r"Action:\s*(\w+)\((.*)\)", output)
        if match:
            fn_name = match.group(1)
            raw_args = match.group(2)
            # Clean arguments
            args = [clean_arg(a) for a in raw_args.split(",")]

            # Execute
            if fn_name in tools:
                observation = tools[fn_name](*args)
                print("Tool called", fn_name, "with args:", args)
            else:
                observation = f"Error: Tool {fn_name} not recognized."

            print(f"\033[92mObservation:\033[0m {observation}\n")
            history += f"\n{output}\nObservation: {observation}"

            if "Confirmed:" in history:
              break

            if "Error:" in history:
              break
        else:
            print("No action found. Ending loop.")
            break

# Test Case
run_agent_lab("Hi, I'm Bob. Book me the cheapest flight from LAX to ORD")



-- Step 1 --
Thought: The user wants to book a flight. First, I need to get the user's information to proceed with the booking.
Action: get_user_info(name="Bob")

Tool called get_user_info with args: ['Bob']
Observation: UserProfile(user_id='2', name='Bob', email='bob@gmail.com')

-- Step 2 --
Thought: Now that I have the user's information, I can fetch the flight information for the requested origin and destination.
Action: fetch_flight_info(origin="LAX", destination="ORD")
[Flight(flight_id='DA701', origin='LAX', destination='ORD', date_time=Date(year=2025, month=11, day=5, hour=6), duration=4, price=300), Flight(flight_id='DA702', origin='LAX', destination='ORD', date_time=Date(year=2025, month=11, day=5, hour=14), duration=4, price=280), Flight(flight_id='DA703', origin='LAX', destination='ORD', date_time=Date(year=2025, month=11, day=6, hour=9), duration=4, price=450)]
Tool called fetch_flight_info with args: ['LAX', 'ORD']
Observation: [Flight(flight_id='DA701', origin='LAX', des

#### **Important**
Please create your Google AI studion API keys as below :
- Visit https://aistudio.google.com/api-keys
- Create Project under "Choose an imported project" dropdown
- Name your key anything you want(Only after creating project)
- Choose create key
- Now you can find key under Key column of the project - click on the key valye to reveal api key
- Copy the key and paste below under "insert_api_key"

**To store your API key securely in Colab secrets:**
1. Click on the 🔑 icon in the left panel.
2. Click 'Add a new secret'.
3. Set the 'Name' to `GOOGLE_API_KEY`.
4. Paste your API key into the 'Value' field.
5. Ensure 'Notebook access' is enabled for this notebook.

Try confusing prompts

In [10]:
run_agent_lab("Hi, I'm Bob. Book me the cheapest flight from Earth to Mars")


-- Step 1 --
Thought: The user wants to book a flight. First, I need to get the user's information to proceed with the booking.
Action: get_user_info(name="Bob")

Tool called get_user_info with args: ['Bob']
Observation: UserProfile(user_id='2', name='Bob', email='bob@gmail.com')

-- Step 2 --
Thought: Now that I have the user's information, I need to find flight information from Earth to Mars.
Action: fetch_flight_info(origin="Earth", destination="Mars")
[]
Tool called fetch_flight_info with args: ['Earth', 'Mars']
Observation: Error: No flights found.



## Acknowledgement

This notebook is inspired from following blog post : https://dspy.ai/tutorials/customer_service_agent/